In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
import torch.onnx as onnx
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.optim as optim

import math
pitorch = torch.Tensor([math.pi])

delt = torch.tensor(1e-5) #step size
Nt = int(1/delt)

## useful functions

In [2]:
def dU(x,y):
    a = torch.tensor([-1,-1,-6.5,0.7])
    b = torch.tensor([0,0,11,0.6])
    c = torch.tensor([-10,-10,-6.5,0.7])
    D = torch.tensor([-200,-100,-170,15])
    X = torch.tensor([1,0,-0.5,-1])
    Y = torch.tensor([0,0.5,1.5,1])
    gamma = torch.tensor([9])
    k = torch.tensor([5])
  
    fx1 = D[0]*torch.exp(a[0]*((x-X[0]).pow(2)) + b[0]*(x-X[0])*(y-Y[0]) + c[0]*(y.pow(2)))
    fx2 = D[1]*torch.exp(a[1]*((x-X[1]).pow(2)) + b[1]*(x-X[1])*(y-Y[1]) + c[1]*((y-Y[1]).pow(2)))
    fx3 = D[2]*torch.exp(a[2]*((x-X[2]).pow(2)) + b[2]*(x-X[2])*(y-Y[2]) + c[2]*((y-Y[2]).pow(2)))
    fx4 = D[3]*torch.exp(a[3]*((x-X[3]).pow(2)) + b[3]*(x-X[3])*(y-Y[3]) + c[3]*((y-Y[3]).pow(2)))
    extra = gamma*torch.sin(2*k*pitorch*x)*torch.sin(2*k*pitorch*y)
    
    extrapx = gamma*torch.cos(2*k*pitorch*x)*torch.sin(2*k*pitorch*y)*2*pitorch*k
    extrapy = gamma*torch.cos(2*k*pitorch*y)*torch.sin(2*k*pitorch*x)*2*pitorch*k
    
    dUx = fx1*(2*a[0]*(x-X[0])+b[0]*(y-Y[0])) + fx2*(2*a[1]*(x-X[1])+b[1]*(y-Y[1])) + fx3*(2*a[2]*(x-X[2])+b[2]*(y-Y[2])) + fx4*(2*a[3]*(x-X[3])+b[3]*(y-Y[3]))
#     dUx = dUx + extrapx
    
    dUy = fx1*(2*c[0]*y+b[0]*(x-X[0])) + fx2*(2*c[1]*(y-Y[1])+b[1]*(x-X[1])) + fx3*(2*c[2]*(y-Y[2])+b[2]*(x-X[2])) + fx4*(2*c[3]*(y-Y[3])+b[3]*(x-X[3]))
#     dUy = dUy + extrapy
#     U = fx1+fx2+fx3+fx4
#     U = U+extra
    
    return dUx, dUy

In [4]:
def chiAB(X):
    a = torch.tensor([-0.558,1.441])
    b = torch.tensor([0.623,0.028])
    r = torch.tensor(0.1)
    m = nn.Tanh()
    sizex, nothing = X.shape
    chiA = 0.5 - 0.5*m(1000*((((X - a).pow(2)).sum(dim = 1).reshape(sizex,1))-(r + torch.tensor(0.02)).pow(2)))                     
    chiB = 0.5 - 0.5*m(1000*((((X - b).pow(2)).sum(dim = 1).reshape(sizex,1))-(r + torch.tensor(0.02)).pow(2)))       
                             
    return chiA, chiB

In [5]:
def q_theta(X,chiA,chiB,q_tilde):
    Q = (torch.tensor([1]) - chiA)*(q_tilde*(torch.tensor([1]) - chiB)+chiB)
    return Q


In [6]:
def funU(x,y):
    a = torch.tensor([-1,-1,-6.5,0.7])
    b = torch.tensor([0,0,11,0.6])
    c = torch.tensor([-10,-10,-6.5,0.7])
    D = torch.tensor([-200,-100,-170,15])
    X = torch.tensor([1,0,-0.5,-1])
    Y = torch.tensor([0,0.5,1.5,1])
    gamma = torch.tensor([9])
    k = torch.tensor([5])
  
    fx1 = D[0]*torch.exp(a[0]*((x-X[0]).pow(2)) + b[0]*(x-X[0])*(y-Y[0]) + c[0]*(y.pow(2)))
    fx2 = D[1]*torch.exp(a[1]*((x-X[1]).pow(2)) + b[1]*(x-X[1])*(y-Y[1]) + c[1]*((y-Y[1]).pow(2)))
    fx3 = D[2]*torch.exp(a[2]*((x-X[2]).pow(2)) + b[2]*(x-X[2])*(y-Y[2]) + c[2]*((y-Y[2]).pow(2)))
    fx4 = D[3]*torch.exp(a[3]*((x-X[3]).pow(2)) + b[3]*(x-X[3])*(y-Y[3]) + c[3]*((y-Y[3]).pow(2)))
    extra = gamma*torch.sin(2*k*pitorch*x)*torch.sin(2*k*pitorch*y)
    
    U = fx1+fx2+fx3+fx4
#     U = U+extra
    
    return U

In [8]:
def funU_bias(x,y,xt,yt):
    w = torch.tensor(5)
    sig = torch.tensor(0.05)
    sig2 = sig.pow(2)
    
    U_bias_vector = w*torch.exp(-(x - xt).pow(2)/(2*sig2) - (y-yt).pow(2)/(2*sig2))
    U_bias = torch.sum(U_bias_vector,0)
    return U_bias

## models and data

In [10]:
class Ruggedmueller2(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        # hidden layer
        self.linear1 = nn.Linear(in_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        # output layer
        self.linear3 = nn.Linear(hidden_size, out_size)
        
    def forward(self, xb):
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        tanhf = nn.Tanh()
        out = tanhf(out)
        # Get predictions using output layer
        out = self.linear2(out)
        out = tanhf(out)
        out = self.linear3(out)
        # apply activation function again
        out = torch.sigmoid(out)
        return out

In [11]:
import pandas as pd
train = torch.tensor([])

df = pd.read_excel('traindata_meta_uniform_in_space_T10.xlsx')

X_train = pd.DataFrame(df['X'])
Y_train = pd.DataFrame(df['Y'])

X_train = torch.tensor(X_train.values,dtype=torch.float32)
Y_train = torch.tensor(Y_train.values,dtype=torch.float32)

train_data = torch.cat((X_train, Y_train), 1)
train_data.requires_grad_(True)

a = torch.tensor([-0.558,1.441]) 
b = torch.tensor([0.623,0.028])

plt.scatter(X_train.detach().numpy(), Y_train.detach().numpy(),s = 0.1)
plt.scatter(a[0],a[1])
plt.scatter(b[0],b[1])
plt.xlim([-1.5,1])
plt.ylim([-0.5,2])

torch.Size([15466, 1])
torch.Size([15466, 1])


tensor([[ 0.0101,  0.0256],
        [-0.0038,  0.0312],
        [-0.0067,  0.0654],
        ...,
        [ 0.9453,  0.2918],
        [ 0.7600, -0.0092],
        [ 0.9273, -0.2069]], requires_grad=True)

In [13]:
# Bumps generated from metadynamics, will be used in the loss function
df = pd.read_excel('data_bump_T10.xlsx')
print(df.head())

X_bump = pd.DataFrame(df['X_bump'])
Y_bump = pd.DataFrame(df['Y_bump'])

X_bump = torch.tensor(X_bump.values,dtype=torch.float32)
Y_bump = torch.tensor(Y_bump.values,dtype=torch.float32)

     X_bump    Y_bump
0  0.000000  0.000000
1  0.364616  0.089371
2  0.473150  0.012352
3  0.536493  0.053063
4  0.541906 -0.041037


In [14]:
input_size = 2
output_size = 1
N_neurons = [10,40]
model = Ruggedmueller2(input_size,N_neurons[1],output_size)

In [34]:
size1,size2 = train_data.shape
rhs = torch.zeros(size1,)
train_ds = TensorDataset(train_data,rhs)
batch_size = int(size1/125) # the batch size is the size of the training data
# int(size1/125)


loss_fn = nn.L1Loss()

train_dl = DataLoader(train_ds, batch_size, shuffle=True)
optimizer1 = optim.Adam(model_11.parameters(), lr=1e-4)

beta = torch.tensor(1/10)

In [1]:
# model_11 training:
for epoch in range(1000):
    if epoch%100 == 0:
    for X,y in train_dl:
        optimizer1.zero_grad()
        chia,chib = chiAB(X)
        q_tilde = model_11(X)
        Q = q_theta(X,chia,chib,q_tilde)
        U = funU(X[:,0],X[:,1])
        derivQ = torch.autograd.grad(Q,X,allow_unused=True, retain_graph=True, grad_outputs = torch.ones_like(Q), create_graph=True)
        output = (torch.norm(derivQ[0],dim=1)**2)*torch.exp(-beta*U)
        loss = loss_fn(output, y)

        loss.backward()
        optimizer1.step()
        
        

In [33]:
torch.save(model_11, 'd-net_T10_2hidden40.pt')